In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import os
import parsers as p
import time
%load_ext autoreload
%autoreload 2

In [72]:
timea=time.time()
df = pd.read_csv("../telemetry_exe/Raw_Data/data0101.csv", dtype = str, on_bad_lines = "skip")
init_sz = df.shape[0]
df = df[df["time"].str.len()==13]
df = df[df['msg.len'].str.len()==1]
df = df[~df['msg.id'].isnull()]
df = df[~(df['msg.id'].str.len()<2)]
df = df[~(df['msg.id'].str.len()>3)]

#df['time'] = pd.to_numeric(df['time'])
#print(np.any(df['time'][1:]-df['time'][:-1]<0))
#indices = np.where(df['time'][1:]-df['time'][:-1]<0)[0]+1
#df.iloc[indices]['time']=df['time'][indices]+1000
#df.iloc[265816-5:265816+5].head(10)

df['time'] = pd.to_datetime(df['time'], unit='ms')
df["msg.id"] = df["msg.id"].apply(int, base = 16)
df["msg.len"] = pd.to_numeric(df['msg.len'])
df = df[df['msg.len']*2 == df['data'].str.len()]
df["data"] = df["data"].apply(int, base = 16)
df["data"] = df['data'].to_numpy(dtype=np.uint64).byteswap()>>(8*(8-df["msg.len"])).to_numpy(dtype=np.uint16)
df = df.reset_index(drop=True)
pruned_sz = df.shape[0]
print(init_sz - pruned_sz)
print(time.time()-timea)

#df.head()

305
1.734907627105713


In [199]:
df = df.sample(frac=1).reset_index(drop=True)
df.sort_values(by=['time'],inplace=True)
df = df.reset_index(drop=True)

In [53]:
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()

In [3]:
unique, freq = np.unique(msg_id, return_counts = True)
hexes = np.array([p.MESSAGE_DICT[i][1] for i in unique])
print(np.array((freq,hexes)).T)

[['36366' 'MC1_status']
 ['35320' 'MC2_status']
 ['35719' 'MC3_status']
 ['35866' 'MC4_status']
 ['34911' 'MC1_temps']
 ['34759' 'MC2_temps']
 ['34847' 'MC3_temps']
 ['34755' 'MC4_temps']
 ['9194' 'MC1_energy']
 ['8694' 'MC2_energy']
 ['8899' 'MC3_energy']
 ['8992' 'MC4_energy']
 ['9237' 'MC1_setpoints_command']
 ['8805' 'MC2_setpoints_command']
 ['9279' 'MC3_setpoints_command']
 ['8892' 'MC4_setpoints_command']
 ['3430' 'MCU_status']
 ['6489' 'BMS_onboard_temperatures']
 ['54322' 'BMS_voltages']
 ['26135' 'BMS_detailed_voltages']
 ['12952' 'BMS_temperatures']
 ['8067' 'BMS_detailed_temperatures']
 ['3355' 'BMS_status']
 ['341' 'Dashboard_status']
 ['3397' 'EM_status']
 ['3409' 'EM_measurement']]


In [7]:
timea=time.time()
df = pd.read_csv("../telemetry_exe/Raw_Data/data0005.csv", dtype = str, on_bad_lines = "skip")
init_sz = df.shape[0]
df = df[df["time"].str.len()==13]
df = df[df['msg.len'].str.len()==1]
df = df[~df['msg.id'].isnull()]
df = df[~(df['msg.id'].str.len()<2)]
df = df[~(df['msg.id'].str.len()>3)]
df['time'] = pd.to_datetime(df['time'], unit='ms')
df["msg.id"] = df["msg.id"].apply(int, base = 16)
df["msg.len"] = pd.to_numeric(df['msg.len'])
df = df[df['msg.len']*2 == df['data'].str.len()]
df["data"] = df["data"].apply(int, base = 16)
df["data"] = df['data'].to_numpy(dtype=np.uint64).byteswap()>>(8*(8-df["msg.len"])).to_numpy(dtype=np.uint16)
df = df.reset_index(drop=True)
pruned_sz = df.shape[0]
print("Removed messages: " + str(init_sz - pruned_sz))
timeb = time.time()
print("CSV parse time: " + str(timeb-timea))
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()
points = 0
messages = 0
for i in p.MESSAGE_DICT.values():
    try:
        a = i[0](msg_id, msg_data)
        points += a.shape[0]
        messages += 1
        print(str(i[1]) + " " + str(a.max()))
    except:
        continue
#print(messages)
#print(points)
print("Message coverage: "+str(points/msg_id.size))
print("Message parse time: " + str(time.time()-timeb))
print("Total time: " + str(time.time()-timea))

#df.head()

Removed messages: 77
CSV parse time: 0.8567438125610352
BMS_detailed_temperatures 167.23
BMS_detailed_voltages 11.0
BMS_onboard_temperatures 167.23
BMS_status 1.0
BMS_temperatures 119.23
BMS_voltages 514.68
Dashboard_status 3.0
MC1_energy 12267.0
MC2_energy 14501.0
MC3_energy 18055.0
MC4_energy 21382.0
MC1_setpoints_command 20000.0
MC2_setpoints_command 20000.0
MC3_setpoints_command 20000.0
MC4_setpoints_command 20000.0
MC1_status 13674.0
MC2_status 13756.0
MC3_status 13630.0
MC4_status 13744.0
MC1_temps 44.1
MC2_temps 42.5
MC3_temps 44.3
MC4_temps 44.5
EM_status 0.0
EM_measurement 0.0
Message coverage: 0.9931872595264821
Message parse time: 0.019840478897094727
Total time: 0.8765842914581299


In [40]:
print(p.parse_MC1_energy(msg_id, msg_data)[:,1].max())
print(p.parse_MC2_energy(msg_id, msg_data)[:,1].max())
print(p.parse_MC3_energy(msg_id, msg_data)[:,1].max())
print(p.parse_MC4_energy(msg_id, msg_data)[:,1].max())

12267.0
14501.0
18055.0
21382.0


In [34]:
p.parse_BMS_detailed_temperatures(msg_id, msg_data)

array([[10.  ,  1.  , 23.65, 26.27,  0.  ],
       [11.  ,  0.  , 23.96, 23.91, 23.71],
       [ 0.  ,  0.  , 23.73, 23.66, 23.6 ],
       ...,
       [10.  ,  1.  , 24.04, 24.06,  0.  ],
       [11.  ,  0.  , 24.29, 24.24, 24.05],
       [11.  ,  1.  , 24.18, 24.06,  0.  ]])

In [2]:
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"

In [18]:
import plotly.graph_objects as go
a = p.parse_BMS_detailed_temperatures(msg_id, msg_data)
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,2], mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,3], mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,4], mode='markers'))
fig.show()

In [38]:
%prun
root = {}

directory = ["a","b","c"]
data = np.array([1,2,3])

cwd = root
for i in directory[:-1]:
    if not root.__contains__(i):
        cwd[i] = {}
    cwd = cwd[i]
cwd[directory[-1]] = data

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [45]:
fig = go.Figure()
a = p.parse_MC1_energy(msg_id, msg_data)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC2_energy(msg_id, msg_data)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC3_energy(msg_id, msg_data)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC4_energy(msg_id, msg_data)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
fig.show()